In [1]:
%%init_spark --stderr
launcher.conf.spark.app.name = "DIC Exercise 2"
launcher.conf.spark.default.parallelism = 12

In [2]:
println("spark.app.name: " + spark.conf.get("spark.app.name"))
println("spark.master: " + spark.conf.get("spark.master"))
println("spark.submit.deployMode: " + spark.conf.get("spark.submit.deployMode"))
println("sc.defaultParallelism: " + sc.defaultParallelism)

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1715326141961_0076
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1715326141961_0076)
SparkSession available as 'spark'


spark.app.name: DIC Exercise 2
spark.master: yarn
spark.submit.deployMode: client
sc.defaultParallelism: 12


In [3]:
!hdfs dfs -ls -h /user/dic24_shared/amazon-reviews/full

Found 2 items


-rw-r--r--   3 blee supergroup      55.3 M 2023-03-13 11:18 /user/dic24_shared/amazon-reviews/full/reviews_devset.json


-rw-r--r--   3 blee supergroup      54.3 G 2023-03-13 11:18 /user/dic24_shared/amazon-reviews/full/reviewscombined.json




In [4]:
//val textFile = sc.textFile("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")

val reviews = spark.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")

[Stage 0:>                                                         (0 + 2) / 12]
[Stage 0:==============>                                           (3 + 2) / 12]
[Stage 0:=================================>                        (7 + 2) / 12]
                                                                                


reviews: org.apache.spark.sql.DataFrame = [asin: string, category: string ... 8 more fields]


In [5]:
reviews.show()

+----------+--------------------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|            category| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------------------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0981850006|Patio_Lawn_and_Garde|  [6, 7]|    5.0|This was a gift f...| 12 3, 2009|A2VNYWOPJ13AFP|Amazon Customer "...|              Delish|    1259798400|
|B00002N66D|Patio_Lawn_and_Garde|  [1, 1]|    5.0|This is a very ni...| 12 3, 2012|A2E5XXXC07AGA7|               James|       Nice spreader|    1354492800|
|B00002N67U|Patio_Lawn_and_Garde|  [0, 1]|    1.0|The metal base wi...|08 13, 2008|A16PX63WZIEQ13|             Finaldx| Terrible spike base|    1218585600|
|B00002N6AN|Patio_Lawn_and_Garde|  [0, 0]|    4.0|For the most p

In [6]:
!pwd

/home/dic24/e12347510/Exercise_2



In [7]:
val stopwordsSource = scala.io.Source.fromFile("/home/dic24/e12347510/Exercise_2/data/stopwords.txt")

// Convert the Source to a Set for efficient lookup
val stopwords = stopwordsSource.getLines.toSet

// Close the Source object to release resources
stopwordsSource.close()

stopwordsSource: scala.io.BufferedSource = <iterator>
stopwords: scala.collection.immutable.Set[String] = Set(serious, latterly, absorbs, looks, particularly, used, e, printer, down, regarding, entirely, regardless, moreover, please, read, ourselves, able, behind, for, despite, s, maybe, viz, further, corresponding, x, any, wherein, across, name, allows, this, instead, in, taste, ought, myself, have, your, off, once, are, is, mon, his, oh, why, rd, knows, bulbs, too, among, course, greetings, somewhat, bibs, everyone, seen, likely, said, try, already, soon, nobody, got, given, song, using, less, am, consider, hence, than, n, accordingly, four, anyhow, want, three, forth, whereby, himself, specify, yes, throughout, inasmuch, but, whether, sure, below, aren, co, best, plus, becomes, what,...


In [8]:
// Extract the reviewText attribute
val reviewTextsDF = reviews.select("reviewText")

// Convert DataFrame to Array[String]
val reviewTextsArray = reviewTextsDF.map(row => row.getString(0)).collect()

[Stage 2:========================>                                 (5 + 3) / 12]
[Stage 2:===========================================>              (9 + 2) / 12]
                                                                                


reviewTextsDF: org.apache.spark.sql.DataFrame = [reviewText: string]
reviewTextsArray: Array[String] = Array(This was a gift for my other husband.  He's making us things from it all the time and we love the food.  Directions are simple, easy to read and interpret, and fun to make.  We all love different kinds of cuisine and Raichlen provides recipes from everywhere along the barbecue trail as he calls it. Get it and just open a page.  Have at it.  You'll love the food and it has provided us with an insight into the culture that produced it. It's all about broadening horizons.  Yum!!, This is a very nice spreader.  It feels very solid and the pneumatic tires give it great maneuverability and handling over bumps.  The control arm is solid metal, not a cable, which gives you precise contro...


In [9]:
def tokenizeAndRemoveDuplicates(reviewText: String): Seq[String] = {
    // Split the review text into tokens using regular expression
    val tokens = reviewText.toLowerCase.split("[^a-zA-Z<>^|]+")

    // Remove duplicates
    tokens.distinct.filter(token => token.length > 1 && !stopwords.contains(token))
}

tokenizeAndRemoveDuplicates: (reviewText: String)Seq[String]


In [10]:
// Apply the function to each review text using map
val tokenizedReviewsRDD = reviewTextsArray.map(tokenizeAndRemoveDuplicates)

tokenizedReviewsRDD: Array[Seq[String]] = Array(WrappedArray(gift, husband, making, things, time, love, food, directions, simple, easy, interpret, make, kinds, cuisine, raichlen, recipes, barbecue, trail, calls, open, page, provided, insight, culture, produced, broadening, horizons, yum), WrappedArray(nice, spreader, feels, solid, pneumatic, tires, give, great, maneuverability, handling, bumps, control, arm, metal, cable, precise, long, time, settings, experimentation, products, true, good, distribution, flings, material, farther, side, left, crappy, edgeguard), WrappedArray(metal, base, hose, attachments, poorly, designed, made, previous, reviewer, pointed, leaks, badly, pops, fix, tighten, junction, plastic, wears, bit, falls, sprinkler, useless, avoid, gilmour, sprinklers, spike, hea...


In [11]:
val totalCount = tokenizedReviewsRDD.map(_.length).sum

totalCount: Int = 2123321


In [12]:
// Extract the category attribute
val categoriesDF = reviews.select("category")

// Convert DataFrame to Array[String]
val categoriesArray = categoriesDF.map(row => row.getString(0)).collect()

categoriesDF: org.apache.spark.sql.DataFrame = [category: string]
categoriesArray: Array[String] = Array(Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn_and_Garde, Patio_Lawn...


In [13]:
val categoryReviewPairsRDD = categoriesArray.zip(tokenizedReviewsRDD)

categoryReviewPairsRDD: Array[(String, Seq[String])] = Array((Patio_Lawn_and_Garde,WrappedArray(gift, husband, making, things, time, love, food, directions, simple, easy, interpret, make, kinds, cuisine, raichlen, recipes, barbecue, trail, calls, open, page, provided, insight, culture, produced, broadening, horizons, yum)), (Patio_Lawn_and_Garde,WrappedArray(nice, spreader, feels, solid, pneumatic, tires, give, great, maneuverability, handling, bumps, control, arm, metal, cable, precise, long, time, settings, experimentation, products, true, good, distribution, flings, material, farther, side, left, crappy, edgeguard)), (Patio_Lawn_and_Garde,WrappedArray(metal, base, hose, attachments, poorly, designed, made, previous, reviewer, pointed, leaks, badly, pops, fix, tighten, junction, plast...


In [14]:
val categoryTermPairsRDD = sc.parallelize(categoryReviewPairsRDD.flatMap {
  case (category, reviewTextArray) =>
    reviewTextArray.map(term => ((category, term),1))
})

categoryTermPairsRDD: org.apache.spark.rdd.RDD[((String, String), Int)] = ParallelCollectionRDD[16] at parallelize at <console>:25


In [15]:
// Group by key (category-term)
val groupedRDD = categoryTermPairsRDD.groupByKey()

// Reduce by key to count the occurrences of each term within each category
val A = groupedRDD.mapValues(iterable => iterable.sum).collect()

24/05/11 17:33:11 WARN TaskSetManager: Stage 4 contains a task of very large size (5489 KiB). The maximum recommended task size is 1000 KiB.

[Stage 4:>                                                         (0 + 1) / 12]
[Stage 4:>                                                         (0 + 2) / 12]
[Stage 4:====>                                                     (1 + 2) / 12]
[Stage 4:=========>                                                (2 + 2) / 12]
[Stage 4:==============>                                           (3 + 2) / 12]
[Stage 4:===================>                                      (4 + 2) / 12]
[Stage 4:========================>                                 (5 + 2) / 12]
[Stage 4:=============================>                            (6 + 2) / 12]
[Stage 4:=================================>                        (7 + 2) / 12]
[Stage 4:======================================>                   (8 + 2) / 12]
[Stage 4:=======================================

groupedRDD: org.apache.spark.rdd.RDD[((String, String), Iterable[Int])] = ShuffledRDD[17] at groupByKey at <console>:25
A: Array[((String, String), Int)] = Array(((Automotive,set),50), ((Electronic,hopes),18), ((Book,salaciously),1), ((Movies_and_TV,sophomoric),1), ((Book,birds),49), ((Electronic,hvr),2), ((Book,marketeering),1), ((Movies_and_TV,sabbath),4), ((Book,mccullough),4), ((Office_Product,funky),2), ((Book,comprehension),22), ((Book,operative),6), ((Book,vengeful),8), ((Movies_and_TV,straight),45), ((Movies_and_TV,cougar),1), ((Book,chap),3), ((Book,gwynne),1), ((Automotive,arerunning),1), ((Kindle_Store,posters),1), ((Movies_and_TV,karajan),1), ((Book,regulation),10), ((Tools_and_Home_Improvement,buyer),4), ((Movies_and_TV,bells),1), ((Book,ot),13), ((Book,native),93), ((Movie...


In [16]:
// Convert the array A to an RDD of key-value pairs
val groupedByTerms = sc.parallelize(A).map { case ((cat, term), value) => (term, value) }

// Group by key (term)
val termGroupedRDD = groupedByTerms.groupByKey()

// Reduce by key to sum up the values for each term
val termTotal = termGroupedRDD.mapValues(_.sum).collect()

[Stage 6:=========>                                                (2 + 2) / 12]
[Stage 6:========================>                                 (5 + 2) / 12]
[Stage 6:======================================>                   (8 + 2) / 12]
                                                                                


groupedByTerms: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[20] at map at <console>:26
termGroupedRDD: org.apache.spark.rdd.RDD[(String, Iterable[Int])] = ShuffledRDD[21] at groupByKey at <console>:29
termTotal: Array[(String, Int)] = Array((vga,32), (mattered,12), (intimately,18), (apostrophes,6), (reunion,46), (bone,92), (valorie,2), (sandberg,1), (qraitians,1), (showy,4), (bombast,3), (folan,1), (hermeneutical,3), (xxoo,1), (dole,1), (adverising,1), (gowns,5), (ducklings,1), (tamanu,1), (ensnare,1), (modifies,2), (iturbide,1), (gayton,1), (omnipresence,1), (crying,98), (boucle,1), (contemptible,3), (semites,1), (surgeplus,1), (husky,17), (artur,3), (merchandising,1), (beauracy,1), (dnr,3), (ranfly,1), (beleive,13), (ior,1), (grant,73), (unitentional,1), (horvatic,1), (...


In [17]:
// Convert A and termTotal to RDDs
val A_RDD = sc.parallelize(A)
val termTotal_RDD = sc.parallelize(termTotal)

// Convert A_RDD and termTotal_RDD to key-value pairs
val A_Keyed_RDD = A_RDD.flatMap { case ((category, term), value) => Some((term, (category, value))) }
val termTotal_Keyed_RDD = termTotal_RDD.map { case (term, value) => (term, value) }

// Calculate the values for each category-term pair in B
val B_RDD = A_Keyed_RDD.mapPartitions(iter => {
  val termTotalMap = iter.map { case (term, (category, valueA)) => (term, valueA) }.toMap
  iter.map { case (term, (category, valueA)) =>
    val valueTotal = termTotalMap.getOrElse(term, 0)
    val valueB = valueTotal - valueA
    ((category, term), (valueA, valueB))
  }
})

A_RDD: org.apache.spark.rdd.RDD[((String, String), Int)] = ParallelCollectionRDD[23] at parallelize at <console>:27
termTotal_RDD: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[24] at parallelize at <console>:28
A_Keyed_RDD: org.apache.spark.rdd.RDD[(String, (String, Int))] = MapPartitionsRDD[25] at flatMap at <console>:31
termTotal_Keyed_RDD: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[26] at map at <console>:32
B_RDD: org.apache.spark.rdd.RDD[((String, String), (Int, Int))] = MapPartitionsRDD[27] at mapPartitions at <console>:35


In [19]:
val A_and_B = B_RDD.collect()

A_and_B: Array[((String, String), (Int, Int))] = Array()
